In [1]:
import numpy as np
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from resnets_utils import *
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline

import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tqdm import tqdm

Using TensorFlow backend.


##  Get images and convert into Numpy array

In [2]:
def load_data():
    train = pd.read_csv('train.csv')
    train_image = []
    for i in tqdm(range(train.shape[0])):
        img = image.load_img('train/'+train['filename'][i], target_size=(28,28,1), grayscale=True)
        img = image.img_to_array(img)
        img = img/255
        train_image.append(img)
    X = np.array(train_image)
    y=train['label'].values
    y = to_categorical(y)
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)
    return X_train, X_test, y_train, y_test
    
    
    

In [3]:
def identity_block(X, f, filters, stage, block):
    """
    Implementation of the identity block as defined in Figure 3
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    
    Returns:
    X -- output of the identity block, tensor of shape (n_H, n_W, n_C)
    """
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value. You'll need this later to add back to the main path. 
    X_shortcut = X
    
    # First component of main path
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
     
    # Second component of main path (≈3 lines)
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X =  Activation('relu')(X)

    # Third component of main path (≈2 lines)
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
      
    return X

In [4]:
tf.reset_default_graph()

with tf.Session() as test:
    np.random.seed(1)
    A_prev = tf.placeholder("float", [3, 4, 4, 6])
    X = np.random.randn(3, 4, 4, 6)
    A = identity_block(A_prev, f = 2, filters = [2, 4, 6], stage = 1, block = 'a')
    test.run(tf.global_variables_initializer())
    out = test.run([A], feed_dict={A_prev: X, K.learning_phase(): 0})
    print("out = " + str(out[0][1][1][0]))

out = [0.19716813 0.         1.3561227  2.1713073  0.         1.3324987 ]


In [17]:
def convolutional_block(X, f, filters, stage, block, s = 2):
    """
    Implementation of the convolutional block as defined in Figure 4
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    s -- Integer, specifying the stride to be used
    
    Returns:
    X -- output of the convolutional block, tensor of shape (n_H, n_W, n_C)
    """
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X


    ##### MAIN PATH #####
    # First component of main path 
    X = Conv2D(F1, (1, 1), strides = (s,s), name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    # Second component of main path (≈3 lines)
    X = Conv2D(F2, (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path (≈2 lines)
    X = Conv2D(F3, (1, 1), strides = (1,1), padding = 'valid',name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    ##### SHORTCUT PATH #### (≈2 lines)
    X_shortcut = Conv2D(F3, (1, 1), strides = (s,s), name = conv_name_base + '1', kernel_initializer = glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
   
    return X

In [6]:
tf.reset_default_graph()

with tf.Session() as test:
    np.random.seed(1)
    A_prev = tf.placeholder("float", [3, 4, 4, 6])
    X = np.random.randn(3, 4, 4, 6)
    A = convolutional_block(A_prev, f = 2, filters = [2, 4, 6], stage = 1, block = 'a')
    test.run(tf.global_variables_initializer())
    out = test.run([A], feed_dict={A_prev: X, K.learning_phase(): 0})
    print("out = " + str(out[0][1][1][0]))

out = [0.09018463 1.2348977  0.46822017 0.0367176  0.         0.65516603]


In [7]:

def ResNet50(input_shape = (64, 64, 3), classes = 6):
    """
    Implementation of the popular ResNet50 the following architecture:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 ->  TOPLAYER

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """
    
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    
    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2), name = 'conv1', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 2, block='a', s = 1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')

    # Stage 3 (≈4 lines)
    X = convolutional_block(X, f = 3, filters = [128, 128, 512], stage = 3, block='a', s = 2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    # Stage 4 (≈6 lines)
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block='a', s = 2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    # Stage 5 (≈3 lines)
    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], stage = 5, block='a', s = 2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    # AVGPOOL (≈1 line). Use "X = AveragePooling2D(...)(X)"
    #X = AveragePooling2D((2, 2), name = 'avg_pool')(X)
    
 
    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet50')

    return model

Run the following code to build the model's graph. If your implementation is not correct you will know it by checking your accuracy when running `model.fit(...)` below.

In [8]:
model = ResNet50(input_shape = (28,28, 1), classes = 10)

## Compile Model

In [9]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

## Prepare train and test data 

In [10]:


X_train, X_test, Y_train, Y_test = load_data()

print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))


  0%|          | 0/49000 [00:00<?, ?it/s]C:\Users\nmadapati\AppData\Local\conda\conda\envs\tf-gpu\lib\site-packages\keras_preprocessing\image.py:492: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '
100%|██████████| 49000/49000 [00:34<00:00, 1407.15it/s]


number of training examples = 39200
number of test examples = 9800
X_train shape: (39200, 28, 28, 1)
Y_train shape: (39200, 10)
X_test shape: (9800, 28, 28, 1)
Y_test shape: (9800, 10)


Run the following cell to train your model on 2 epochs with a batch size of 32. On a CPU it should take you around 5min per epoch. 

In [ ]:
model.fit(X_train, Y_train, epochs = 50, batch_size = 32)


Epoch 1/50


 6400/39200 [===>..........................] - ETA: 1:06:57 - loss: 0.0408 - acc: 1.00 - ETA: 1:04:58 - loss: 0.0636 - acc: 0.98 - ETA: 1:03:16 - loss: 0.1119 - acc: 0.96 - ETA: 1:02:32 - loss: 0.1480 - acc: 0.95 - ETA: 1:03:43 - loss: 0.2170 - acc: 0.93 - ETA: 1:04:03 - loss: 0.1907 - acc: 0.94 - ETA: 1:04:11 - loss: 0.1869 - acc: 0.95 - ETA: 1:03:57 - loss: 0.1951 - acc: 0.94 - ETA: 1:03:26 - loss: 0.1943 - acc: 0.94 - ETA: 1:03:23 - loss: 0.2005 - acc: 0.93 - ETA: 1:03:17 - loss: 0.1972 - acc: 0.93 - ETA: 1:02:55 - loss: 0.2035 - acc: 0.93 - ETA: 1:02:34 - loss: 0.2011 - acc: 0.93 - ETA: 1:02:13 - loss: 0.1970 - acc: 0.93 - ETA: 1:01:57 - loss: 0.1913 - acc: 0.93 - ETA: 1:02:09 - loss: 0.2051 - acc: 0.92 - ETA: 1:02:16 - loss: 0.1973 - acc: 0.92 - ETA: 1:02:24 - loss: 0.2030 - acc: 0.92 - ETA: 1:02:45 - loss: 0.2071 - acc: 0.93 - ETA: 1:02:55 - loss: 0.2021 - acc: 0.93 - ETA: 1:03:25 - loss: 0.2031 - acc: 0.93 - ETA: 1:03:22 - loss: 0.1994 - acc: 0.93 - ETA: 1:03:27 - loss: 0.2381 -

12928/39200 [========>.....................] - ETA: 53:12 - loss: 0.2440 - acc: 0.93 - ETA: 53:08 - loss: 0.2434 - acc: 0.93 - ETA: 53:05 - loss: 0.2432 - acc: 0.93 - ETA: 53:01 - loss: 0.2430 - acc: 0.93 - ETA: 52:59 - loss: 0.2450 - acc: 0.93 - ETA: 52:56 - loss: 0.2454 - acc: 0.93 - ETA: 52:52 - loss: 0.2449 - acc: 0.93 - ETA: 52:48 - loss: 0.2447 - acc: 0.93 - ETA: 52:46 - loss: 0.2447 - acc: 0.93 - ETA: 52:42 - loss: 0.2444 - acc: 0.93 - ETA: 52:39 - loss: 0.2442 - acc: 0.93 - ETA: 52:35 - loss: 0.2441 - acc: 0.93 - ETA: 52:32 - loss: 0.2445 - acc: 0.93 - ETA: 52:28 - loss: 0.2462 - acc: 0.93 - ETA: 52:24 - loss: 0.2454 - acc: 0.93 - ETA: 52:22 - loss: 0.2452 - acc: 0.93 - ETA: 52:18 - loss: 0.2455 - acc: 0.93 - ETA: 52:14 - loss: 0.2444 - acc: 0.93 - ETA: 52:11 - loss: 0.2450 - acc: 0.93 - ETA: 52:07 - loss: 0.2442 - acc: 0.93 - ETA: 52:04 - loss: 0.2437 - acc: 0.93 - ETA: 52:01 - loss: 0.2462 - acc: 0.93 - ETA: 51:58 - loss: 0.2452 - acc: 0.93 - ETA: 51:55 - loss: 0.2450 - acc: 

19456/39200 [=============>................] - ETA: 42:10 - loss: 0.2420 - acc: 0.93 - ETA: 42:07 - loss: 0.2415 - acc: 0.93 - ETA: 42:03 - loss: 0.2426 - acc: 0.93 - ETA: 42:01 - loss: 0.2430 - acc: 0.93 - ETA: 41:58 - loss: 0.2430 - acc: 0.93 - ETA: 41:55 - loss: 0.2426 - acc: 0.93 - ETA: 41:52 - loss: 0.2427 - acc: 0.93 - ETA: 41:49 - loss: 0.2421 - acc: 0.93 - ETA: 41:46 - loss: 0.2418 - acc: 0.93 - ETA: 41:42 - loss: 0.2414 - acc: 0.93 - ETA: 41:39 - loss: 0.2409 - acc: 0.93 - ETA: 41:36 - loss: 0.2420 - acc: 0.93 - ETA: 41:33 - loss: 0.2422 - acc: 0.93 - ETA: 41:30 - loss: 0.2423 - acc: 0.93 - ETA: 41:26 - loss: 0.2424 - acc: 0.93 - ETA: 41:23 - loss: 0.2424 - acc: 0.93 - ETA: 41:20 - loss: 0.2428 - acc: 0.93 - ETA: 41:17 - loss: 0.2431 - acc: 0.93 - ETA: 41:15 - loss: 0.2430 - acc: 0.93 - ETA: 41:12 - loss: 0.2429 - acc: 0.93 - ETA: 41:09 - loss: 0.2427 - acc: 0.93 - ETA: 41:05 - loss: 0.2423 - acc: 0.93 - ETA: 41:02 - loss: 0.2424 - acc: 0.93 - ETA: 40:59 - loss: 0.2419 - acc: 

25984/39200 [==================>...........] - ETA: 31:41 - loss: 0.2366 - acc: 0.93 - ETA: 31:38 - loss: 0.2373 - acc: 0.93 - ETA: 31:35 - loss: 0.2371 - acc: 0.93 - ETA: 31:32 - loss: 0.2369 - acc: 0.93 - ETA: 31:29 - loss: 0.2369 - acc: 0.93 - ETA: 31:26 - loss: 0.2374 - acc: 0.93 - ETA: 31:23 - loss: 0.2371 - acc: 0.93 - ETA: 31:20 - loss: 0.2367 - acc: 0.93 - ETA: 31:17 - loss: 0.2364 - acc: 0.93 - ETA: 31:14 - loss: 0.2361 - acc: 0.93 - ETA: 31:11 - loss: 0.2360 - acc: 0.93 - ETA: 31:08 - loss: 0.2358 - acc: 0.93 - ETA: 31:05 - loss: 0.2361 - acc: 0.93 - ETA: 31:02 - loss: 0.2359 - acc: 0.93 - ETA: 30:59 - loss: 0.2356 - acc: 0.93 - ETA: 30:56 - loss: 0.2354 - acc: 0.93 - ETA: 30:53 - loss: 0.2351 - acc: 0.93 - ETA: 30:49 - loss: 0.2348 - acc: 0.93 - ETA: 30:47 - loss: 0.2346 - acc: 0.93 - ETA: 30:43 - loss: 0.2346 - acc: 0.93 - ETA: 30:40 - loss: 0.2344 - acc: 0.93 - ETA: 30:38 - loss: 0.2357 - acc: 0.93 - ETA: 30:34 - loss: 0.2357 - acc: 0.93 - ETA: 30:31 - loss: 0.2358 - acc: 

28448/39200 [====================>.........] - ETA: 21:10 - loss: 0.2281 - acc: 0.94 - ETA: 21:07 - loss: 0.2279 - acc: 0.94 - ETA: 21:04 - loss: 0.2277 - acc: 0.94 - ETA: 21:01 - loss: 0.2276 - acc: 0.94 - ETA: 20:58 - loss: 0.2276 - acc: 0.94 - ETA: 20:55 - loss: 0.2277 - acc: 0.94 - ETA: 20:52 - loss: 0.2277 - acc: 0.94 - ETA: 20:49 - loss: 0.2274 - acc: 0.94 - ETA: 20:46 - loss: 0.2272 - acc: 0.94 - ETA: 20:44 - loss: 0.2271 - acc: 0.94 - ETA: 20:40 - loss: 0.2270 - acc: 0.94 - ETA: 20:37 - loss: 0.2268 - acc: 0.94 - ETA: 20:34 - loss: 0.2266 - acc: 0.94 - ETA: 20:31 - loss: 0.2271 - acc: 0.94 - ETA: 20:28 - loss: 0.2271 - acc: 0.94 - ETA: 20:25 - loss: 0.2268 - acc: 0.94 - ETA: 20:22 - loss: 0.2281 - acc: 0.94 - ETA: 20:19 - loss: 0.2281 - acc: 0.94 - ETA: 20:16 - loss: 0.2279 - acc: 0.94 - ETA: 20:13 - loss: 0.2276 - acc: 0.94 - ETA: 20:10 - loss: 0.2278 - acc: 0.94 - ETA: 20:06 - loss: 0.2280 - acc: 0.94 - ETA: 20:03 - loss: 0.2280 - acc: 0.94 - ETA: 20:01 - loss: 0.2278 - acc: 

Let's see how this model (trained on only two epochs) performs on the test set.

In [12]:
preds = model.evaluate(X_test, Y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

9800/9800 [==============================] - ETA: 6: - ETA: 4: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 

In [13]:
preds


[0.21399107929243116, 0.931734693877551]

## Prepare test data

In [19]:
test = pd.read_csv('test.csv')
test_image = []
for i in tqdm(range(test.shape[0])):
    img = image.load_img('test/'+test['filename'][i], target_size=(28,28,1), grayscale=True)
    img = image.img_to_array(img)
    img = img/255
    test_image.append(img)
X_test = np.array(test_image)


100%|██████████| 21000/21000 [00:07<00:00, 2775.31it/s]


## Predict test data

In [20]:

predict_y = model.predict(X_test)


## Create submission File

In [21]:
submission = pd.DataFrame({ "filename": test['filename'],      
                            "label": np.argmax(predict_y, axis=1) })
    

submission.to_csv('Digit_Submission1.csv', index=False)

